# Building an ETL Pipeline

In [18]:
import pandas as pd
import mysql.connector 
import pymysql
from sqlalchemy import create_engine
from datetime import datetime, date, time, timedelta
import tweepy
import numpy as np 
import pyodbc
from sqlalchemy import create_engine
import urllib

### Connecting to Twitter

In [7]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [19]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

In [20]:
def twitter_df():
    api = twitter_setup()
    all_tweets = []
    tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
    df.set_index('Tweets')
    return df
twitter_df=twitter_df()

In [21]:
def extract_municipality_hashtags(df):
    new_df = pd.DataFrame([])
    municipality_dict = { '@CityofCTAlerts' : 'Cape Town',
            '@CityPowerJhb' : 'Johannesburg',
            '@eThekwiniM' : 'eThekwini' ,
            '@EMMInfo' : 'Ekurhuleni',
            '@centlecutility' : 'Mangaung',
            '@NMBmunicipality' : 'Nelson Mandela Bay',
            '@CityTshwane' : 'Tshwane'}
  
    length = len(df['Tweets'])
    mun_list = []
    flag = 0
    for row in df['Tweets']:
        flag = 0
        for key in municipality_dict.keys():
            if key in row:
               mun_list.append(municipality_dict[key])
               flag = 1
            break
        if not flag:
          mun_list.append(np.nan)

    final_hash = []
    for row in df['Tweets']:
     final_hash.append([string for string in row.lower().split() if string[0][0] == '#'])
    final_hash = [np.nan if x == [] else x for x in final_hash]
    
    new_df['Tweets'] = df['Tweets']
    new_df['Date'] = df['Date']
    new_df['municipality'] = mun_list
    new_df['hashtags'] = final_hash
    
    return new_df
extract_municipality_hashtags(twitter_df())

TypeError: 'DataFrame' object is not callable

In [11]:
# host = COMPUTERNAME\SQLEXPRESS

params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=MANDLA63\SQLEXPRESS;DATABASE=gather_eskom;UID=sa;trusted_connection='true'")
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

df = twitter_df
df.to_sql('twitter_table', con=engine, if_exists='append')

In [12]:
# Check that temp table is in your MS SQL Database
conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                     user='sa')

twitter_df = pd.read_sql_query('select * from dbo.twitter_table',conn)
twitter_df.head()

,index,Tweets,Date
0,0,Interfering with or vandalising Eskom property...,2020-03-06 06:34:00
1,1,#POWERALERT 1\n\nDate: 06 March 2020\n\nNo loa...,2020-03-06 05:54:58
2,2,#Eskom #MediaStatement\n\nEskom notes allegati...,2020-03-05 19:28:53
3,3,DYK that you can report #poweroutages on MyEsk...,2020-03-05 16:00:00
4,4,#POWERALERT 2\nDate: 05 March 2020\n\nNo loads...,2020-03-05 15:32:36


### Connecting Data Base

In [13]:
df2

NameError: name 'df2' is not defined

### SOURCE CODE TO BE ALTERED

In [14]:
def MySQLConnect(self,query):
        """
        Connects to database and extracts
        raw tweets and any other columns we
        need
        Parameters:
        ----------------
        arg1: string: SQL query
        Returns: pandas dataframe
        ----------------
        """

        try:
            con = mysql.connector.connect(host = self.host, database = self.database, \
                user = self.user, password = self.password, charset = 'utf8')

            if con.is_connected():
                print("Successfully connected to database")

                cursor = con.cursor()
                query = query
                cursor.execute(query)

                data = cursor.fetchall()
                # store in dataframe
                df = pd.DataFrame(data,columns = ['date', 'tweet'])
                #print(df.head())



        except Error as e:
            print(e)

        cursor.close()
        con.close()

        
        return df #None